In [1]:
import os
import numpy as np
import cv2
import math

def ci95(inp):
    max95 = np.mean(inp) + (1.96 * (np.std(inp) / math.sqrt(len(inp))))
    min95 = np.mean(inp) - (1.96 * (np.std(inp) / math.sqrt(len(inp))))
    return min95, max95

def dice_coef_score(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
#     print(intersection)
    return (2. * intersection) / (np.sum(y_true_f) + np.sum(y_pred_f))


In [6]:
name_li = ['brmh','kyuh','gugh']
cutoff_values=[0.3,0.4,0.5]

for cutoff_value in cutoff_values:
    print('-'*20)
    print('cutoff : {}'.format(cutoff_value))
    print('-'*20)
    for name in name_li:
        print(name+'_respiratory')
        gt_base = 'D:/lunit/01_data_analysis/data/img/result/{}_respiratory/gt_mask'.format(name)
        base = 'D:/lunit/01_data_analysis/data/img/ai_json(all_files)/{}_respiratory/ai_mask'.format(name)
        ai_mask_list = os.listdir(base)
        sum_image = len(ai_mask_list)

        fppi_cnt = 0
        fn_cnt = 0
        tp_cnt = 0

        total_cnt=0
        gt_ims= []
        ai_ims = []
        true_detection_rate_li = []
        for index, ai_mask in enumerate(sorted(ai_mask_list)):
    #         if index==10:
    #             break
            tp = 0
            fn = 0
            gt_file = '{}_mask.png'.format(ai_mask.split('_cutoff')[0])
            gt_im = cv2.imread(os.path.join(gt_base, gt_file))
            w,h,_=gt_im.shape
            im = cv2.imread(os.path.join(base, ai_mask))
            im = cv2.resize(im, (h,w),interpolation=cv2.INTER_LINEAR)
    #         print(gt_file, ai_mask)
    #         print(gt_im.shape, im.shape)

            imgray = cv2.cvtColor(gt_im,cv2.COLOR_BGR2GRAY)
            ret, thresh = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY)
            gt_contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
            ret, thresh = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY)
            ai_contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #         print(len(gt_contours), len(ai_contours))
            total_cnt+=len(ai_contours)
        #     print(gt_im.shape, im.shape)
            if gt_im.sum() == 0: # normal
                fppi_cnt += len(ai_contours)

            elif im.sum()==0:
                fn_cnt += len(gt_contours)
                fn+=1

            else:
                for ai_contour in ai_contours:
                    _image = np.zeros_like(gt_im)
                    _image_ai = cv2.drawContours(_image, [ai_contour], 0, color=1, thickness=-1)
    #                 if _image[gt_im>0].sum() == 0:
    #                 dsc = dice_coef_score(gt_im/255, _image)
    #                 if dsc<cutoff_value:
    #                     print(dsc)
    #                     fppi_cnt += 1
                    dsc_li=[]
                    for gt_contour in gt_contours:
                        _image = np.zeros_like(gt_im)
                        _image_gt = cv2.drawContours(_image, [gt_contour], 0, color=1, thickness=-1)

        #                 if _image[im>0].sum() == 0:
    #                     print(dice_coef_score(_image_gt, _image_ai))
                        dsc_li.append(dice_coef_score(_image_gt, _image_ai))

                    if max(dsc_li) < cutoff_value:
                        fppi_cnt += 1
                    else:
                        tp_cnt+=1

                for gt_contour in gt_contours:
                    _image = np.zeros_like(gt_im)
                    _image_gt = cv2.drawContours(_image, [gt_contour], 0, color=1, thickness=-1)

    #                 if _image[gt_im>0].sum() == 0:
    #                 dsc = dice_coef_score(gt_im/255, _image)
    #                 if dsc<cutoff_value:
    #                     print(dsc)
    #                     fppi_cnt += 1
                    dsc_li=[]
                    for ai_contour in ai_contours:
                        _image = np.zeros_like(gt_im)
                        _image_ai = cv2.drawContours(_image, [ai_contour], 0, color=1, thickness=-1)
        #                 if _image[im>0].sum() == 0:
    #                     print(dice_coef_score(_image_gt, _image_ai))
                        dsc_li.append(dice_coef_score(_image_ai, _image_gt))

                    if max(dsc_li) < cutoff_value:
                        fn_cnt += 1

    #         if tp+fn>0:
    #             true_detection_rate = tp/(tp+fn)
    #             true_detection_rate_li.append(true_detection_rate)
    #         else:
    #             true_detection_rate_li.append(1)

    #         print(total_cnt, tp_cnt, fn_cnt)        
        #     print("intermediate sum ffpi cnt: {}".format(fppi_cnt))



        fppi = fppi_cnt/sum_image
        print('='*20)
        print('Calculated FPPI: {:0.2f}'.format(fppi))
        print('Total count: {}'.format(total_cnt))
        print('TP count: {}'.format(tp_cnt))
        print('FP count: {}'.format(fppi_cnt))
        print('FN count: {}'.format(fn_cnt))
        print('Sensitivity: {:0.2f}'.format(tp_cnt/(tp_cnt+fn_cnt)))
        print('='*20)
    #     true_detection_rate = tp_cnt/(tp_cnt+fn_cnt)
    #     min_ci, max_ci = ci95(np.array(true_detection_rate_li))
    #     print('true_detection_rate: {:0.2f} ({:0.2f}, {:0.2f})'.format(true_detection_rate, min_ci, max_ci))


--------------------
cutoff : 0.3
--------------------
brmh_respiratory
Calculated FPPI: 0.51
Total count: 2650
TP count: 1361
FP count: 1289
FN count: 619
Sensitivity: 0.69
kyuh_respiratory
Calculated FPPI: 0.46
Total count: 2061
TP count: 1145
FP count: 916
FN count: 785
Sensitivity: 0.59
gugh_respiratory
Calculated FPPI: 0.46
Total count: 1648
TP count: 969
FP count: 679
FN count: 403
Sensitivity: 0.71
--------------------
cutoff : 0.4
--------------------
brmh_respiratory
Calculated FPPI: 0.56
Total count: 2650
TP count: 1237
FP count: 1413
FN count: 752
Sensitivity: 0.62
kyuh_respiratory
Calculated FPPI: 0.51
Total count: 2061
TP count: 1051
FP count: 1010
FN count: 892
Sensitivity: 0.54
gugh_respiratory
Calculated FPPI: 0.51
Total count: 1648
TP count: 894
FP count: 754
FN count: 479
Sensitivity: 0.65
--------------------
cutoff : 0.5
--------------------
brmh_respiratory
Calculated FPPI: 0.62
Total count: 2650
TP count: 1086
FP count: 1564
FN count: 909
Sensitivity: 0.54
kyuh_re

# FPPI 계산 (CI 구간 X)

In [5]:
import os
import numpy as np
import cv2

names = ['brmh_healthcheck','kyuh_healthcheck','gugh_healthcheck']
for name in names:
    gt_base = 'D:/lunit/data/img/result/'+name+'/gt_mask'
    base = 'D:/lunit/data/img/ai_json(all_files)/'+name+'/ai_mask'
    ai_mask_list = os.listdir(base)
    sum_image = len(ai_mask_list)

    fppi_cnt = 0
    for index, ai_mask in enumerate(sorted(ai_mask_list)):
        # if index==10:
        #     break

        gt_file = '{}_mask.png'.format(ai_mask.split('_cutoff')[0])
        gt_im = cv2.imread(os.path.join(gt_base, gt_file), cv2.IMREAD_ANYDEPTH | cv2.IMREAD_GRAYSCALE)
        w,h=gt_im.shape
    #     print(h,w)
    #     print("started reading gt file: {}".format(gt_file))

        im = cv2.imread(os.path.join(base, ai_mask))
        im = cv2.resize(im, (h,w))
        imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY)
        ai_contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        if gt_im.sum() == 0: # normal
            fppi_cnt += len(ai_contours)

        else:
            for ai_contour in ai_contours:
                _image = np.zeros_like(gt_im)
                _image = cv2.drawContours(_image, [ai_contour], 0, color=255, thickness=-1)
                if _image[gt_im>0].sum() == 0:
                    fppi_cnt += 1
    #     print("intermediate sum ffpi cnt: {}".format(fppi_cnt))

    print(name)
    print('Calculated FPPI: {}'.format(fppi_cnt/sum_image))

brmh_healthcheck
Calculated FPPI: 0.3547815820543093
kyuh_healthcheck
Calculated FPPI: 0.17987152034261242
gugh_healthcheck
Calculated FPPI: 0.6666666666666666
